In [1]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words_nl=set(nltk.corpus.stopwords.words('dutch'))
stop_words_en=set(nltk.corpus.stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Irina\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
import os
import re

texts = []

for file in os.listdir("data"):
    if re.match(r"\d*\.txt", file):
        with open(f"data/{file}") as text:
            lines = text.readlines()[1:]
            remove_n = []
            for line in lines:
                remove_n.append(line.strip())

            joined_lines = ''.join(remove_n)
            texts.append(joined_lines)

print(len(texts))
print(texts[0])

In [32]:
import spacy
import nl_core_news_lg

nlp = spacy.load("nl_core_news_lg", disable=["parser", "ner"])
en_words = set(nltk.corpus.words.words())
overly_abundant = ["artikel", "kunst", "publicatie"]
months = ["januari", "februari", "maart", "april", "mei", "juni", "juli", "augustus", "september", "november", "december"]

def lemmatizer(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    texts = [text.replace("\n", " ").strip() for text in texts]
    texts = [text.replace(";", " ").strip() for text in texts]
    texts = [text.replace(",", " ").strip() for text in texts]
    texts = [text.replace(".", " ").strip() for text in texts]
    docs = nlp.pipe(texts)
    for token in docs:
        print(token.text, token.pos_, token.dep_)
        if token.pos_ in allowed_postags:
            cleaned_lemmas = [[t.lemma_ for t in doc if t.lemma_ not in stop_words_nl 
                    and t.lemma_ in en_words
                    and t.lemma_ not in stop_words_nl
                    and t.lemma_.lower() not in overly_abundant
                    and t.lemma_.lower() not in months
                    and not re.match(r".*\d.*", t.lemma_)] for doc in docs]
    

    return cleaned_lemmas

In [33]:
cleaned = []
parts = int(len(texts)/20)
for part in range(parts):
    print(f"Batch {part + 1} of {parts}")
    cleaned.append(lemmatizer(texts[20*part:20*(part+1)]))
#lemmatizer(texts)
print(len(cleaned))
print(cleaned[0][0])

Batch 1 of 38


AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'pos_'

In [5]:

print(len(cleaned[0]))

20


# Start

In [24]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [26]:
print(type(cleaned))

<class 'list'>


In [29]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("nl_core_news_lg", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(texts)
print (lemmatized_texts)

KeyboardInterrupt: 

In [28]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(cleaned)


TypeError: decoding to str: need a bytes-like object, list found

In [25]:
bigrams_phrases = gensim.models.Phrases(cleaned, min_count=5, threshold=30)
trigrams_phrases = gensim.models.Phrases(bigrams_phrases[cleaned], threshold=30)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigrams_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])
    
def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

TypeError: unhashable type: 'list'

# end

In [9]:
vect = TfidfVectorizer(stop_words=stop_words_en,max_features=1000)

In [10]:
cleaned_joined = []
    
[[cleaned_joined.append(" ".join(words)) for words in batch] for batch in cleaned]
print(len(cleaned_joined))

760


In [8]:
len(cleaned_joined)

760

In [12]:
vectorised = vect.fit_transform(cleaned_joined)
vect.get_feature_names()

['ab',
 'abstract',
 'accountant',
 'actor',
 'ad',
 'addendum',
 'adjunct',
 'administration',
 'administrative',
 'adverse',
 'agenda',
 'agent',
 'aid',
 'al',
 'alain',
 'alcohol',
 'alexander',
 'aller',
 'also',
 'alt',
 'alter',
 'alternance',
 'alternative',
 'ambulance',
 'ambulant',
 'amount',
 'amusement',
 'analyse',
 'angst',
 'anne',
 'antecedent',
 'antwerp',
 'apart',
 'appeal',
 'appear',
 'applicable',
 'application',
 'arbiter',
 'argument',
 'arm',
 'arrest',
 'art',
 'article',
 'artiste',
 'ascendent',
 'ask',
 'aspect',
 'aspirant',
 'assessed',
 'assessment',
 'assessor',
 'assumer',
 'assurer',
 'astrid',
 'attent',
 'attest',
 'attraction',
 'audit',
 'auditoria',
 'august',
 'aura',
 'aus',
 'australian',
 'authority',
 'auto',
 'autobus',
 'autocar',
 'baby',
 'bachelor',
 'bail',
 'band',
 'bandagist',
 'bank',
 'banket',
 'bar',
 'barbara',
 'base',
 'based',
 'basis',
 'bateau',
 'bed',
 'begin',
 'beman',
 'benelux',
 'berat',
 'bernard',
 'bert',
 'best

In [13]:
idf=vect.idf_
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0:10])

['art', 'minister', 'wet', 'lid', 'twee', 'state', 'dag', 'elk', 'datum', 'decreet']


## LDA

In [15]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 

lda_top=lda_model.fit_transform(vectorised)

In [16]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lda_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
nederlands wet lid art minister extra student school divers per 

Topic 1: 
lid wet art minister decreet twee state divers dag per 

Topic 2: 
centrum accountant code art lid sector dossier vier raad bier 

Topic 3: 
centrum minister kind art wet lid model state correct decreet 

Topic 4: 
lid wet prospectus den twee art kind minister arrest extern 

Topic 5: 
brussels minister art wet sector lid state chauffeur crisis dag 

Topic 6: 
per wet lid decreet art kind centrum post information extra 

Topic 7: 
overleg wet programma lid minister context week art dag basis 

Topic 8: 
den das wird ist sind es um hat art muss 

Topic 9: 
decreet minister sport media art lid kind brussels kans twee 



## BERTopic

In [17]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True)
topics, probs = topic_model.fit_transform(cleaned_joined)

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 392kB/s]
Downloading: 100%|██████████| 190/190 [00:00<00:00, 95.1kB/s]
Downloading: 100%|██████████| 10.2k/10.2k [00:00<00:00, 1.70MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 16.6kB/s]
Downloading: 100%|██████████| 39.3k/39.3k [00:00<00:00, 389kB/s]
Downloading: 100%|██████████| 349/349 [00:00<00:00, 116kB/s]
Downloading: 100%|██████████| 90.9M/90.9M [00:04<00:00, 21.6MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 13.3kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 37.4kB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 1.03MB/s]
Downloading: 100%|██████████| 350/350 [00:00<00:00, 87.4kB/s]
Downloading: 100%|██████████| 13.2k/13.2k [00:00<00:00, 826kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 819kB/s] 


In [18]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,0,534,0_lid_wet_art_minister
1,1,106,1_brussels_art_minister_state
2,2,83,2_den_wird_das_ist
3,3,19,3_overleg_context_week_dag
4,4,18,4____


In [19]:
topic_model.get_topic(0) 

[('lid', 0.13480193161896628),
 ('wet', 0.12053301424245846),
 ('art', 0.1031708779462964),
 ('minister', 0.08863461942837322),
 ('decreet', 0.08721541569722849),
 ('twee', 0.0809182113830681),
 ('per', 0.055315869099044054),
 ('dag', 0.05209650628063224),
 ('state', 0.05035613014944304),
 ('elk', 0.048038312305715945)]

In [20]:
topic_model.visualize_topics()

In [21]:
topic_model.visualize_distribution(probs[200], min_probability=0.0015)

In [22]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [23]:
topic_model.visualize_barchart(top_n_topics=5)